In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from config import gkey

In [6]:
data_address = "../data/cities.csv"

weather_df = pd.read_csv(data_address)
weather_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,0,chuy,uy,-33.6971,-53.4616,73.26,56,0,12.46
1,1,new norfolk,au,-42.7826,147.0587,44.80,55,62,1.99
2,2,hithadhoo,mv,-0.6000,73.0833,80.83,64,0,3.09
3,3,dillon,us,34.4166,-79.3712,53.87,35,1,19.57
4,4,saldanha,za,-33.0117,17.9442,69.80,68,0,9.22


In [7]:
weather_clean_df = weather_df.loc[weather_df["Humidity"] < 100]

In [8]:
gmaps.configure(api_key=gkey)

In [9]:
locations = weather_clean_df[["Lat", "Lng"]].astype(float)

humidity_per = weather_clean_df["Humidity"].astype(float)

print(max(humidity_per))

98.0


In [14]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_per,
                                dissipating=False, max_intensity=max(humidity_per),
                                point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
ideal_df = weather_clean_df.loc[(weather_clean_df["Temperature (F)"] > 75) & 
                                (weather_clean_df["Temperature (F)"] < 100) & 
                                (weather_clean_df["Wind Speed (mph)"] < 10) & 
                                (weather_clean_df["Wind Speed (mph)"] > 5) &
                               (weather_clean_df["Cloudiness"] < 5)]
ideal_df.reset_index().drop(["index", "Unnamed: 0"], axis=1).head()

,City,Country,Lat,Lng,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,lazaro cardenas,mx,17.9583,-102.2000,85.23,51,0,7.07
1,villa carlos paz,ar,-31.4241,-64.4978,85.98,43,0,7.00
2,mount isa,au,-20.7333,139.5000,82.40,34,0,8.05
3,necochea,ar,-38.5473,-58.7368,87.01,40,0,5.66
4,san luis,ar,-33.2950,-66.3356,87.58,17,0,5.30


In [29]:
print(len(ideal_df["City"]))

13
